In [30]:
import pandas as pd
import numpy as np
import logging
from typing import List, Optional

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from lifelines import KaplanMeierFitter
from lifelines.utils import restricted_mean_survival_time, median_survival_times

In [2]:
dtype_map = pd.read_csv('../outputs/final_df_dtypes.csv', index_col = 0).iloc[:, 0].to_dict()
df = pd.read_csv('../outputs/final_df.csv', dtype = dtype_map)

In [3]:
df.shape

(6461, 149)

In [4]:
df_death = pd.read_csv('../outputs/full_cohort_with_death_data.csv', dtype = dtype_map)

In [6]:
df = pd.merge(df, df_death[['PatientID', 'LineName', 'event', 'duration']], on = 'PatientID', how = 'left')

In [7]:
df.shape

(6461, 152)

In [8]:
df['treatment'] = np.where(df['LineName'] == 'chemo', 0, 1)

In [9]:
df.columns.to_list()

['PatientID',
 'DiseaseGrade',
 'SmokingStatus',
 'Surgery',
 'GroupStage_mod',
 'TStage_mod',
 'NStage_mod',
 'MStage_mod',
 'SurgeryType_mod',
 'days_diagnosis_to_adv',
 'adv_diagnosis_year',
 'days_diagnosis_to_surgery',
 'PrimarySite_lower',
 'Gender',
 'age',
 'Ethnicity_mod',
 'Race_mod',
 'region',
 'PDL1_status',
 'FGFR_status',
 'PDL1_binary',
 'ecog_index',
 'ecog_newly_gte2',
 'weight_index',
 'bmi_index',
 'percent_change_weight',
 'hypotension',
 'tachycardia',
 'fevers',
 'hypoxemia',
 'albumin',
 'alp',
 'alt',
 'ast',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'hemoglobin',
 'platelet',
 'potassium',
 'sodium',
 'total_bilirubin',
 'wbc',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_max',
 'bicarbonate_max',
 'bun_max',
 'calcium_max',
 'chloride_max',
 'creatinine_max',
 'hemoglobin_max',
 'platelet_max',
 'potassium_max',
 'sodium_max',
 'total_bilirubin_max',
 'wbc_max',
 'albumin_min',
 'alp_min',
 'alt_min',
 'ast_min',
 'bicarbonate_min',
 '

In [10]:
cat_var = ['ecog_index', 'GroupStage_mod']
cont_var = ['age', 'creatinine', 'albumin']
passthrough_var = ['Surgery', 'ecog_newly_gte2']

In [11]:
# Build pipeline
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', RobustScaler())
])

categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown = 'ignore'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_pipeline, cont_var),
        ('cat', categorical_pipeline, cat_var),
        ('pass', 'passthrough', passthrough_var)],
        remainder = 'drop'
)

# Fit and transform
X_preprocessed = preprocessor.fit_transform(df)

In [12]:
# Logistic regression 
lr_model = LogisticRegression(class_weight = 'balanced')
lr_model.fit(X_preprocessed, df['treatment'])

LogisticRegression(class_weight='balanced')

In [13]:
# Logistic regression 
lr_model = LogisticRegression(class_weight = 'balanced')
lr_model.fit(X_preprocessed, df['treatment'])
propensity_score = lr_model.predict_proba(X_preprocessed)[:, 1]
df['propensity_score'] = propensity_score
        
# Calculate unstabilized weights if stabilized = False
df['weight'] = (
    np.where(df['treatment'] == 1, 
             1/df['propensity_score'], 
             1/(1 - df['propensity_score']))
)

In [15]:
all_var = cat_var + cont_var + passthrough_var

In [16]:
all_var

['ecog_index',
 'GroupStage_mod',
 'age',
 'creatinine',
 'albumin',
 'Surgery',
 'ecog_newly_gte2']

In [17]:
smd_df = df[all_var + ['treatment', 'weight']].copy()

In [18]:
smd_df

,ecog_index,GroupStage_mod,age,creatinine,albumin,Surgery,ecog_newly_gte2,treatment,weight
0,2,IV,80,1.23,42.0,1,0,1,1.617371
1,unknown,unknown,77,1.18,41.0,1,0,0,1.883739
2,unknown,unknown,81,1.45,36.0,0,0,1,1.754244
3,unknown,IV,79,NaN,NaN,0,0,0,1.509234
4,0-1,0-II,67,1.10,45.0,0,0,1,1.735440
...,...,...,...,...,...,...,...,...,...
6456,0-1,IV,63,1.30,34.0,0,0,0,1.275862
6457,unknown,unknown,59,0.82,42.0,1,0,0,1.210022
6458,0-1,0-II,84,0.90,41.0,0,0,1,1.226059
6459,0-1,unknown,73,1.10,28.0,1,0,1,1.832493


In [21]:
treat_mask = df['treatment'] == 1
control_mask = df['treatment'] == 0

In [49]:
import warnings
from lifelines.exceptions import StatisticalWarning

In [50]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=StatisticalWarning)
    treat_km.fit(
        durations = df.loc[treat_mask, 'duration'],
        event_observed = df.loc[treat_mask, 'event'],
        weights = df.loc[treat_mask, 'weight']
    )

In [46]:
# Estimate survival times
# Kaplan-Meier models 
treat_km = KaplanMeierFitter()
control_km = KaplanMeierFitter()

treat_mask = df['treatment'] == 1
control_mask = df['treatment'] == 0

treat_km.fit(
    durations = df.loc[treat_mask, 'duration'],
    event_observed = df.loc[treat_mask, 'event'],
    weights = df.loc[treat_mask, 'weight']
)

control_km.fit(
    durations = df.loc[control_mask, 'duration'],
    event_observed = df.loc[control_mask, 'event'],
    weights = df.loc[control_mask, 'weight']
)

# Initialize empty dictionaries 
results = {
    'treatment': {},
    'control': {},
    'difference': {}
}

psurv_time_points = [180]
rmst_time_points = [365]
median_time = True

# Probability of survival at selected time points
if psurv_time_points is not None:
    results['treatment']['survival_prob'] = {}
    results['control']['survival_prob'] = {}
    results['difference']['survival_prob'] = {}
    
    for t in psurv_time_points:
        treat_surv = treat_km.predict(t)
        control_surv = control_km.predict(t)
        results['treatment']['survival_prob'][t] = treat_surv
        results['control']['survival_prob'][t] = control_surv
        results['difference']['survival_prob'][t] = treat_surv - control_surv

# RMST calculation at selected time points
if rmst_time_points is not None: 
    results['treatment']['rmst'] = {}
    results['control']['rmst'] = {}
    results['difference']['rmst'] = {}

    for t in rmst_time_points:
        treat_rmst = restricted_mean_survival_time(treat_km, t=t)
        control_rmst = restricted_mean_survival_time(control_km, t=t)
        results['treatment']['rmst'][t] = treat_rmst
        results['control']['rmst'][t] = control_rmst
        results['difference']['rmst'][t] = treat_rmst - control_rmst

# Median survival time calculations
if median_time: 
    treat_med = treat_km.median_survival_time_
    control_med = control_km.median_survival_time_
    results['treatment']['median'] = treat_med
    results['control']['median'] = control_med
    results['difference']['median'] = treat_med - control_med

/Users/xavierorcutt/Dropbox/TrialRescue/myenv/lib/python3.13/site-packages/lifelines/fitters/kaplan_meier_fitter.py:338: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/xavierorcutt/Dropbox/TrialRescue/myenv/lib/python3.13/site-packages/lifelines/fitters/kaplan_meier_fitter.py:338: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances

In [47]:
results

{'treatment': {'survival_prob': {180: np.float64(0.5952671132722624)},
  'rmst': {365: np.float64(242.13320884421313)},
  'median': np.float64(281.0)},
 'control': {'survival_prob': {180: np.float64(0.7501627141020287)},
  'rmst': {365: np.float64(283.01154124829463)},
  'median': np.float64(392.0)},
 'difference': {'survival_prob': {180: np.float64(-0.15489560082976628)},
  'rmst': {365: np.float64(-40.878332404081505)},
  'median': np.float64(-111.0)}}

In [ ]:
# Calculate bootstrapped 95% CIs
# Initialize empty disctionaries for bootstrapped results 
boot_results = {
'treatment': {'survival_prob': {}, 'rmst': {}, 'median': []},
'control': {'survival_prob': {}, 'rmst': {}, 'median': []},
'difference': {'survival_prob': {}, 'rmst': {}, 'median': []}
}

# Loop over n_bootstrap
rng = np.random.default_rng(seed = 42)
for i in range(n_bootstrap):

# Sample with replacement using random indices
indices = rng.choice(df.index, size = len(df), replace = True)
df_boot = df.loc[indices].reset_index(drop=True)

# Recalculate weights using saved model spec
df_boot_weighted = self.fit_transform(
    df_boot,
    treatment_col = self.treatment_col,
    cat_var = self.cat_var,
    cont_var = self.cont_var,
    binary_var = self.binary_var,
    stabilized = self.stabilized,
    lr_kwargs = self.lr_kwargs,
    clip_bounds = self.clip_bounds
)

# Kaplan-Meier models 
treat_km = KaplanMeierFitter()
control_km = KaplanMeierFitter()

treat_mask = df_boot_weighted[self.treatment_col] == 1
control_mask = df_boot_weighted[self.treatment_col] == 0

treat_km.fit(
    durations = df_boot_weighted.loc[treat_mask, self.duration_col],
    event_observed = df_boot_weighted.loc[treat_mask, self.event_col],
    weights = df_boot_weighted.loc[treat_mask, weight_col]
)

control_km.fit(
    durations = df_boot_weighted.loc[control_mask, self.duration_col],
    event_observed = df_boot_weighted.loc[control_mask, self.event_col],
    weights = df_boot_weighted.loc[control_mask, weight_col]
)

# Calculate survival metrics
if psurv_time_points is not None:
    for t in psurv_time_points:
        boot_results['treatment']['survival_prob'].setdefault(t, []).append(treat_km.predict(t))
        boot_results['control']['survival_prob'].setdefault(t, []).append(control_km.predict(t))
        boot_results['difference']['survival_prob'].setdefault(t, []).append(
            treat_km.predict(t) - control_km.predict(t)
        )

if rmst_time_points is not None:
    for t in rmst_time_points:
        treat_rmst = restricted_mean_survival_time(treat_km, t=t)
        control_rmst = restricted_mean_survival_time(control_km, t=t)
        boot_results['treatment']['rmst'].setdefault(t, []).append(treat_rmst)
        boot_results['control']['rmst'].setdefault(t, []).append(control_rmst)
        boot_results['difference']['rmst'].setdefault(t, []).append(treat_rmst - control_rmst)

if median_time:
    treat_med = treat_km.median_survival_time_
    control_med = control_km.median_survival_time_
    boot_results['treatment']['median'].append(treat_med)
    boot_results['control']['median'].append(control_med)
    boot_results['difference']['median'].append(treat_med - control_med)
